## Meta program
- https://www.youtube.com/watch?v=sPiWg5jSoZI&t=632s

In [2]:
from functools import wraps, partial
import os

# No arguments
# One level
def debug(func):
    if "DEBUG" not in os.environ:
        return func
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

### With arguments

In [3]:
# Two level
def debug(prefix=""):
    def decorate(func):
        msg = prefix + func.__qualname__
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)
        return wrapper
    return decorate
# Not using two levels
def debug(func=None, *, prefix=''):
    if fun is None:
        # Wasn't passed
        return partial(debug, prefix=prefix)
    msg = prefix + func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

### Class decorators
- Only instance methods get wrapped
- Why?

In [5]:
# classmethod and staticmethod will not be wrapped
def debugmethods(cls):
    # cls is a class
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, debug(val))
    return cls

### Change method of class

In [7]:
# Rewriting part of the class itself
def debugattr(cls):
    orig_getattribute = cls.__getattribute__
    
    def __getattribute__(self, name):
        print('Get:', name)
        return orig_getattribute(self, name)
    cls.__getattribute__ = __getattribute__
    return cls

### A Metaclass

- debug all the classes

In [8]:
class debugmeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        clsobj = debugmethods(clsobj)
        return clsobj

- Usage

In [11]:
class Base(metaclass=debugmeta):
    pass
class Spam(Base):
    pass
# In python2

class Base(object):
    __metaclss__ = debugmeta
    pass
# OR in file
__metaclss__ = debugmeta

### What happens during class defination
- Step 1:Body of class is isolated

In [13]:
body = '''
def __init__(self, name):
    self.name = name
def bar(self):
    print("I'm Spam.bar")
'''

- Step 2:The class dictionary is created

In [16]:
# This dictionary serves as local namespace for statements 
# in class body, by default it's a simple dictionary(more later)

clsdict = type.__prepare__('Spam', (Base,))

- Step3:Body is executed in returned dict
- Afterwards, clsdict is polulated

In [18]:
exec(body, globals(), clsdict)

- Step 4:Class is constructed from its name, base classes, and the dictionary

In [26]:
Spam = type('Spam', (Base, ), clsdict)

### Defining a New Metaclass
- You typically inherit from type and redefine `__new__` or `__init__`

In [28]:
class mytype(type):
    def __new__(cls, name, bases, clsdict):
        clsobj = super().__new__(cls, name, bases, clsdict)
        return clsobj

- Metaclasses get information about class definitions at the time of definition
    - Can inspect this data
    - Can modify this data
- Similar to a class decorator
- Metaclass changes the meta at the time the class definitions, decorators after this procedure

### Why use it?
- Metaclasses propagated down hierarchies like a genetic mutation

In [30]:
class Base(metaclass=mytype):
    pass
class Spam(Base):    # metaclass=mytype
    pass
class Grok(Spam):    # metaclass=mytype
    pass

- Setted in the base, and debug the universe
    - Debugging gets applied across entire hierarchy
    - Implicitly applied in subclass

### Big picture

- Decorators: Functions
- Class decorators: Classes
- Metaclasses: Class hierarchies

In [32]:
# Not support for keyword args
# Missing calling signatures
class Structure:
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Wrong # args')
        for name, val in zip(self._fields, args):
            setattr(self, name, val)
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

- Solve muti data creatures

In [33]:
from inspect import Parameter, Signature

def make_signature(names):
    return Signature(Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
                    for name in names)
class Structure:
    __signature__ = make_signature([])
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

- To use...

In [35]:
class Point(Structure):
    __signature__ = make_signature(['x', 'y'])

- Use class decorators
- Type checking doen't work

In [36]:
def add_signature(*names):
    def decorate(cls):
        cls.__signature__ = make_signature(names)
        return cls
    return decorate
# Usage
@add_signature('names', 'shares', 'price')
class Stock(Structure):
    pass
@add_signature('x', 'y')
class Point(Structure):
    pass

- Metaclass Solution

In [38]:
class StructMeta(type):
    def __new__(cls, name, bases, clsdict):
        clsobj = super().__new__(cls, name, bases, clsdict)
        sig = make_signature(clsobj._fields)
        setattr(clsobj, '__signature__', sig)
        return clsobj
# Usage
class Structure(metaclass=StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

### Tho Dot
- A basic descriptor

In [39]:
class Descriptor:
    def __init__(self, name=None):
        self.name = name
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
    def __delete__(self, instance):
        del instance.__dict__[self.name]

- Type checking

In [40]:
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Expected %s' % self.ty)
        super().__set__(instance, value)